In [1]:
import numpy as np
from sklearn.datasets import make_blobs

In [3]:
N = 100
np.random.seed(0)
X = np.sort(np.random.rand(N), axis=0)
y = 3 * X - 0.5
y = y + 0.2 * np.random.randn(N)

In [5]:
import torch
import torch.nn as nn
X = torch.from_numpy(X).reshape(-1, 1).float()
y = torch.from_numpy(y).float()

In [6]:
class DualSVR():
    def __init__(self, n_samples, eps=0.5, C=10, gamma=100, iters=50000, lr=1e-5):
        self.n_samples = n_samples
        self.eps = eps
        self.C = C
        self.gamma = gamma
        self.alpha_1 = nn.Parameter(torch.randn(n_samples).abs())
        self.alpha_2 = nn.Parameter(torch.randn(n_samples).abs())
        self.iters = iters
        self.lr = lr
    
    def _loss(self, x, y):
        loss1 = eps * (self.alpha_1 + self.alpha_2).sum()
        diff = self.alpha_1 - self.alpha_2
        loss2 = 0.5 * (torch.outer(diff, diff) * (torch.mm(X, X.T))).sum()
        loss3 = - torch.dot(diff, y)
        loss4 = self.gamma * diff.sum() ** 2
        loss = loss1 + loss2 + loss3 + loss4
        return loss
    
    def _zero_grad(self):
        if self.alpha_1.grad is not None:
            self.alpha_1.grad.zero_()
            self.alpha_2.grad.zero_()
    
    def _update_weights(self):
        self.alpha_1.data -= self.lr * self.alpha_1.grad.data
        self.alpha_2.data -= self.lr * self.alpha_2.grad.data
    
    def _clip_weights(self):
        self.alpha_1.data = torch.clamp(self.alpha_1.data, min=0, max=self.C)
        self.alpha_2.data = torch.clamp(self.alpha_2.data, min=0, max=self.C)
    
    def fit(self, X, y):
        for i in range(self.iters):
            self._zero_grad()
            self._clip_weights()
            loss = self._loss(X, y)
            loss.backward()
            self._update_weights()
        
        with torch.no_grad():
            self._clip_weights()
            diff = self.alpha_1 - self.alpha_2
            self.w = torch.matmul(X.T, diff)
            idx1 = torch.where(self.alpha_1>0)[0]
            b1 = (y[idx1] - torch.matmul(X[idx1], self.w) - self.eps).min()
            idx2 = torch.where(self.alpha_2>0)[0]
            b2 = (y[idx2] - torch.matmul(X[idx2], self.w) + self.eps).max()
            self.b = (b1 + b2)/2
    
    def predict(self, data):    
        with torch.no_grad():
            pred = torch.matmul(data, self.w) + self.b
        
        return pred

In [7]:
eps = 0.1
C = 10
X = X.reshape(-1, 1)
N = len(X)
clf = DualSVR(N, eps=eps, C=C, gamma=100, iters=100000)

In [8]:
clf.fit(X, y)

In [9]:
pred = clf.predict(X)